In [ ]:
import tensorflow as tf
from tensorflow import keras

In [ ]:
# load imdb dataset using tensforflow datasets
import tensorflow_datasets as tfds
datasets, info = tfds.load("imdb_reviews", as_supervised=True, with_info=True)
train_size = info.splits["train"].num_examples
test_size = info.splits["test"].num_examples

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteF3FWG1/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteF3FWG1/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteF3FWG1/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [ ]:
# define a preprocessing function that only uses tensorflow transformations
def preprocess(X_batch, y_batch):
    X_batch = tf.strings.substr(X_batch, 0, 300)
    X_batch = tf.strings.regex_replace(X_batch, rb"<br\s*/?>", b" ")
    X_batch = tf.strings.regex_replace(X_batch, b"[^a-zA-Z']", b" ")
    X_batch = tf.strings.split(X_batch)
    return X_batch.to_tensor(default_value=b"<pad>"), y_batch

In [ ]:
# construct dataset "vocabulary" by applying the preprocess function to each word and then counting the frequency
from collections import Counter

vocabulary = Counter()
for X_batch, y_batch in datasets["train"].batch(32).map(preprocess):
    for review in X_batch:
        vocabulary.update(list(review.numpy()))

vocabulary.most_common()[:3] # most common words

vocab_size = 10000 # truncate the vocabulary to just the most frequent 10000 words
truncated_vocabulary = [
    word for word, count in vocabulary.most_common()[:vocab_size]]

In [ ]:
# replace each word with it's ID using a lookup table
words = tf.constant(truncated_vocabulary)
word_ids = tf.range(len(truncated_vocabulary), dtype=tf.int64)
vocab_init = tf.lookup.KeyValueTensorInitializer(words, word_ids)
num_oov_buckets = 1000
table = tf.lookup.StaticVocabularyTable(vocab_init, num_oov_buckets)

In [ ]:
# create a final training dataset using the lookup table from above and prefetch
def encode_words(X_batch, y_batch):
    return table.lookup(X_batch), y_batch

train_set = datasets["train"].repeat().batch(32).map(preprocess)
train_set = train_set.map(encode_words).prefetch(1)

In [ ]:
# create and train the model
embed_size = 128
model = keras.models.Sequential([
    keras.layers.Embedding(vocab_size + num_oov_buckets, embed_size,
                           mask_zero=True, # not shown in the book
                           input_shape=[None]),
    keras.layers.GRU(128, return_sequences=True),
    keras.layers.GRU(128),
    keras.layers.Dense(1, activation="sigmoid")
])
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
history = model.fit(train_set, steps_per_epoch=train_size // 32, epochs=5)

Epoch 1/5
781/781 [==============================] - 23s 30ms/step - loss: 0.5333 - accuracy: 0.7244
Epoch 2/5
781/781 [==============================] - 24s 30ms/step - loss: 0.3418 - accuracy: 0.8579
Epoch 3/5
781/781 [==============================] - 24s 31ms/step - loss: 0.1898 - accuracy: 0.9326
Epoch 4/5
781/781 [==============================] - 24s 30ms/step - loss: 0.1325 - accuracy: 0.9532
Epoch 5/5
781/781 [==============================] - 24s 30ms/step - loss: 0.1005 - accuracy: 0.9635


In [ ]:
model.save("sentimentAnalysisModel")
scores = model()

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


INFO:tensorflow:Assets written to: sentimentAnalysisModel/assets


INFO:tensorflow:Assets written to: sentimentAnalysisModel/assets


In [ ]:
import tensorflow as tf
from tensorflow import keras

# load imdb test dataset using tensforflow datasets
import tensorflow_datasets as tfds
datasets, info = tfds.load("imdb_reviews", as_supervised=True, with_info=True)
test_size = info.splits["test"].num_examples

# define a preprocessing function that only uses tensorflow transformations
def preprocess(X_batch, y_batch):
    X_batch = tf.strings.substr(X_batch, 0, 300)
    X_batch = tf.strings.regex_replace(X_batch, rb"<br\s*/?>", b" ")
    X_batch = tf.strings.regex_replace(X_batch, b"[^a-zA-Z']", b" ")
    X_batch = tf.strings.split(X_batch)
    return X_batch.to_tensor(default_value=b"<pad>"), y_batch

model = keras.models.load_model("sentimentAnalysisModel")

test_set = datasets["test"].repeat().batch(32).map(preprocess)
test_set = test_set.map(encode_words).prefetch(1)

def reviewScores(model, reviews):
  